In [1]:
from ase import Atoms, Atom
from ase.io import read, write
from ase.data import chemical_symbols, covalent_radii, vdw_alvarez
from ase.parallel import paropen as open
from gpaw import GPAW, PW, FermiDirac, LCAO
# from gpaw.new.ase_interface import GPAW
from gpaw import Davidson
from gpaw import Mixer, MixerSum, MixerDif
from gpaw.directmin.etdm_lcao import LCAOETDM
from gpaw.cdft.cdft import CDFT
from pathlib import Path
import os
import numpy as np
from pymatgen.core import Element
from tqdm.auto import tqdm
import pandas as pd

In [ ]:

magnetism = 'NM'

# for symbol in tqdm(chemical_symbols):
for symbol in tqdm(['H', 'C', 'N', 'O', 'Li', 'Be', 'Mg', 'Na', 'Cr', 'Fe', 'Mo', 'Cu', 'B', 'Si', 'As', 'Ge']):
    
    s = set([symbol])
    
    if 'X' in s:
        continue
        
    try:
        atom = Atom(symbol)
        rmin = covalent_radii[atom.number] * 2 * 0.6
        rvdw = vdw_alvarez.vdw_radii[atom.number] if atom.number < len(vdw_alvarez.vdw_radii) else np.nan 
        rmax = 3.1 * rvdw if not np.isnan(rvdw) else 6
        rstep = 0.2 #if rmin < 1 else 0.4

        a = max(2 * rmax, 15)

        da = symbol + symbol

        # out_dir = Path(str(da + f"_{magnetism}"))
        out_dir = Path(str(da))
        os.makedirs(out_dir, exist_ok=True)

        npts = int((rmax - rmin)/rstep)
        rs = np.linspace(rmin, rmax, npts)
        e = np.zeros_like(rs)

        r = rs[-1]

        positions = [
            [a/2-r/2, a/2, a/2],
            [a/2+r/2, a/2, a/2],
        ]

        try:
            element = Element(symbol)
            m = element.valence[1]
            if element.valence == (0, 2):
                m = 0
        except:
            m = 0

        skip = 0

        traj_fpath = out_dir / "PBE.extxyz"

        if traj_fpath.exists():
            traj = read(traj_fpath, index=":")
            skip = len(traj)
            atoms = traj[-1]
        else:
            # Create the unit cell with two atoms
            atoms = Atoms(
                da, 
                positions=positions,
                # magmoms=magmoms,
                cell=[a, a+0.001, a+0.002], 
                pbc=True
            )

        # if magnetism == 'FM':
        #     if m == 0:
        #         continue
        #     magmoms = [m, m]
        # elif magnetism == 'AFM':
        #     if m == 0:
        #         continue
        #     magmoms = [m, -m]
        # elif magnetism == 'NM':
        #     magmoms = [0, 0]



        print(atoms)

        restart_fpath = out_dir / 'restart.gpw'

        calc = GPAW(
            mode=PW(2000),
            xc='PBE',
            spinpol=True,
            # basis='dzp'
            basis='szp(dzp)',
            # h=0.25,
            # nbands=0 if element.is_noble_gas else '110%',
            hund=False,
            mixer=MixerDif(0.01, 1, 1) if element.is_transition_metal else MixerDif(0.25, 3, 10),
            # eigensolver='rmm-diis', #Davidson(3),  # This solver can parallelize over bands Davidson(3), #
            occupations=FermiDirac(0.2, fixmagmom=False), # if not element.is_metal else FermiDirac(0.2, fixmagmom=False),
            # eigensolver=LCAOETDM(),
            #     # searchdir_algo={'name': 'l-bfgs-p', 'memory': 10}),
            # occupations={'name': 'fixed-uniform'},
            # mixer={'backend': 'no-mixing'},
            # nbands='nao',
            symmetry={'point_group': True},
            txt=out_dir / 'out.txt',
            convergence={
                'eigenstates': 1e-5,
                'density': 5e-3,
                'energy': 5e-4,
                # 'bands': 4
            },
            # {'energy': 0.0005,  # eV / electron
            #  'density': 1.0e-4,  # electrons / electron
            #  'eigenstates': 4.0e-8,  # eV^2 / electron
            #  'bands': 'occupied'},
            # parallel={'gpu': True},
            setups='paw'  
        )
        # calc = GPAW(
        #     mode='pw', #PW(1500),
        #     xc='PBE',
        #     # spinpol=True,
        #     # basis='szp(dzp)',
        #     # h=0.25,
        #     # nbands=0 if element.is_noble_gas else '110%',
        #     hund=False,
        #     # mixer=MixerDif(0.01, 1, 1) if element.is_transition_metal else MixerDif(0.25, 3, 10),
        #     # eigensolver='rmm-diis', #Davidson(3),  # This solver can parallelize over bands Davidson(3), #
        #     # occupations=FermiDirac(0.01, fixmagmom=False), # if not element.is_metal else FermiDirac(0.2, fixmagmom=False),
        #     # eigensolver=LCAOETDM(),
        #     #     # searchdir_algo={'name': 'l-bfgs-p', 'memory': 10}),
        #     # occupations={'name': 'fixed-uniform'},
        #     # mixer={'backend': 'no-mixing'},
        #     # nbands='nao',
        #     # symmetry={'point_group': True},
        #     txt=out_dir / 'out.txt',
        #     convergence={
        #         'eigenstates': 1e-5,
        #         'density': 5e-3,
        #         'energy': 5e-4,
        #         # 'bands': 4
        #     },
        #     # {'energy': 0.0005,  # eV / electron
        #     #  'density': 1.0e-4,  # electrons / electron
        #     #  'eigenstates': 4.0e-8,  # eV^2 / electron
        #     #  'bands': 'occupied'},
        #     parallel={'gpu': True},
        #     setups='paw'
        # )
        # calc.attach(calc.write, 10, restart_fpath, mode='all')

        atoms.calc = calc

        # cdft = CDFT(calc=calc, atoms=atoms, spinspin_regions=      
        # atoms.calc = cdft

        for i, r in enumerate(tqdm(np.flip(rs))):

            if i < skip:
                continue

            positions = [
                [a/2-r/2, a/2, a/2],
                [a/2+r/2, a/2, a/2],
            ]

            atoms.set_positions(positions)

            if i == 0:
                fm_magmoms = [m, m]
                atoms.set_initial_magnetic_moments(fm_magmoms) 
                fm_energy = atoms.get_potential_energy()
                # shutil.move(out_dir / "WAVECAR", out_dir / "WAVECAR_FM")

                afm_magmoms = [m, -m]
                atoms.set_initial_magnetic_moments(afm_magmoms) 
                afm_energy = atoms.get_potential_energy()
                # shutil.move(out_dir / "WAVECAR", out_dir / "WAVECAR_AFM")

                if fm_energy <= afm_energy:
                    magmoms = fm_magmoms
                    atoms.set_initial_magnetic_moments(magmoms) 
                    # shutil.move(out_dir / "WAVECAR_FM", out_dir / "WAVECAR")       
                else:
                    magmoms = afm_magmoms
                    atoms.set_initial_magnetic_moments(magmoms)
                    # shutil.move(out_dir / "WAVECAR_AFM", out_dir / "WAVECAR")

#             if i > 0:   
#                 magmoms = atoms.get_magnetic_moments()

#             atoms.set_initial_magnetic_moments(magmoms)
                # m = min(abs(magmoms[0])*1.2, m)
                # magmoms = magmoms*m/np.abs(magmoms)

            e[i] = atoms.get_potential_energy()

            atoms.calc.results.update({
                "forces": atoms.get_forces()
            })

            write(traj_fpath, atoms, append=True)
    except Exception as e:
        print(e)


  0%|          | 0/16 [00:00<?, ?it/s]

Atoms(symbols='H2', pbc=True, cell=[15.0, 15.001, 15.002], initial_magmoms=..., calculator=SinglePointCalculator(...))


  0%|          | 0/16 [00:00<?, ?it/s]

Atoms(symbols='C2', pbc=True, cell=[15.0, 15.001, 15.002], initial_magmoms=..., calculator=SinglePointCalculator(...))


  0%|          | 0/22 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Atoms(symbols='Na2', pbc=True, cell=[15.5, 15.501, 15.502])


  0%|          | 0/28 [00:00<?, ?it/s]

Did not converge!  See text output for help.
Atoms(symbols='Cr2', pbc=True, cell=[15.190000000000001, 15.191, 15.192000000000002], initial_magmoms=..., calculator=SinglePointCalculator(...))


  0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
!echo $GPAW_NEW

In [ ]:


df = pd.DataFrame(columns=['name', 'method', 'R', 'E', 'F', 'S^2'])



for symbol in tqdm(chemical_symbols):
    
    for magnetism in ['AFM', 'FM', 'NM']:
    
        da = symbol + symbol

        # out_dir = Path(da)
        out_dir = Path(str(da + f"_{magnetism}"))

        traj_fpath = out_dir / "traj.extxyz"

        if traj_fpath.exists():
            traj = read(traj_fpath, index=":")
        else:
            continue

        Rs, Es, Fs, S2s = [], [], [], []
        for atoms in traj:

            vec = atoms.positions[1] - atoms.positions[0]
            r = np.linalg.norm(vec)
            e = atoms.get_potential_energy()
            # f = np.inner(vec/r, atoms.get_forces()[1])
            # s2 = np.mean(np.power(atoms.get_magnetic_moments(), 2))

            Rs.append(r)
            Es.append(e)
            # Fs.append(f)
            # S2s.append(s2)

        data = {
            'name': da,
            'method': f'GGA-PBE (GPAW): {magnetism}',
            'R': Rs,
            'E': Es,
            'F': Fs,
            'S^2': S2s
        }

        df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

json_fpath = 'homonuclear-diatomics.json'

df.to_json(json_fpath, orient='records')    